In [1]:
import pandas as pd
import numpy as np
import os
import re
import nltk
from nltk.stem import RSLPStemmer

In [2]:
def orengo(song):
    st = RSLPStemmer()
    words = filter(lambda x: len(x)>0, song)
    word_list = list(map(lambda x: st.stem(x), words))
    return word_list

In [3]:
def remove_stopwords(doc):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    new_doc = list(filter(lambda x: x not in stopwords, doc))
    return new_doc

In [4]:
df = pd.read_csv('.\data\data.csv')

In [5]:
df.head()

,artist,music_name,lyrics,translated,foreign
0,/marilia-mendonca/,/marilia-mendonca/ausencia.html,sei bem o que te faz bem eu sei mas no fundo e...,sei bem o que te faz bem eu sei mas no fundo e...,2
1,/marilia-mendonca/,/marilia-mendonca/todo-mundo-vai-sofrer.html,a garrafa precisa do copo o copo precisa da me...,a garrafa precisa do copo o copo precisa da me...,5
2,/marilia-mendonca/,/marilia-mendonca/bebi-liguei.html,acordei mais uma vez embriagado e o seu cheiro...,acordei mais uma vez embriagado e o seu cheiro...,22
3,/marilia-mendonca/,/marilia-mendonca/ciumeira.html,no começo eu entendia mas era só cama não tinh...,no começo eu entendia era só cama não tinha am...,11
4,/marilia-mendonca/,/marilia-mendonca/bem-pior-que-eu.html,mente que vai dar uma volta e vem me ver entre...,mente que vai dar uma volta e vem me ver entre...,3


In [16]:
df[pd.isnull(df).any(axis=1)]

,artist,music_name,lyrics,translated,foreign
298,/anitta/,/anitta/juego.html,no yo no soy igual no te vaya' a equivocar ten...,NaN,286
310,/anitta/,/anitta/get-to-know-me-with-alesso.html,you wanna skip the conversation sayin' you're ...,NaN,332
616,/gabriela-rocha/,/gabriela-rocha/aleluia-hallelujah.html,aleluia hallelujah aleluia hallelujah aleluia ...,NaN,64
742,/ana-carolina/,/ana-carolina/encostar-na-tua.html,encostar na tua encostar na tua encostar na tu...,NaN,96
756,/ana-carolina/,/ana-carolina/10-minutos.html,minutos minutos minutos minutos minutos minuto...,NaN,35
757,/ana-carolina/,/ana-carolina/problemas.html,problemas problemas problemas problemas proble...,NaN,32


In [11]:
df.iloc[293,2]

'que quede claro si tú te portas bien yo te lo voy a dar como nunca te lo han dado no te equivoques cuida el terreno no sabes en lo que te estás metiéndo una vez que tu siembres en otro suelo porque yo soy tu veneno controlando tu cuerpo tú me das lo que quiero soy tu veneno estás jugando con fuego cuidao que te quemo porque yo soy tu veneno controlando tu cuerpo tú me das lo que quiero soy tu veneno estás jugando con fuego ten cuidao que te quemo que pienses más en mi de lo que pienso en ti tratas de mantenerte en el juego y te dejo seguir aún no tienes claro que soy la que controla pero dale vamos paso a paso que me va elevando el calor tu pantalón va guayando y ya te tengo pensando porque yo soy tu veneno controlando tu cuerpo tú me das lo que quiero soy tu veneno estás jugando con fuego cuidao que te quemo porque yo soy tu veneno controlando tu cuerpo tú me das lo que quiero soy tu veneno estás jugando con fuego ten cuidao que te quemo que quede claro si tú te portas bien yo te lo 

In [10]:
df.iloc[293,3]

'dado tú cuidao que te quemo tú quemo seguir pero dale vamos paso a paso tú cuidao que te quemo tú quemo dado tú cuidao que te quemo tú q'

In [6]:
df = df.fillna('')

In [7]:
#creates new df where the column lyrics carries every song of a given artist
df1 = df.groupby('artist', sort=False).agg({'foreign': 'sum', 'lyrics': lambda x: ' '.join(x), 'translated': lambda x: ' '.join(x)}).reset_index()

In [8]:
df1.head()

,artist,foreign,lyrics,translated
0,/marilia-mendonca/,412,sei bem o que te faz bem eu sei mas no fundo e...,sei bem o que te faz bem eu sei mas no fundo e...
1,/roberto-carlos/,512,eu tenho tanto pra lhe falar mas com palavras ...,eu tenho tanto pra lhe falar mas com palavras ...
2,/gusttavo-lima/,320,ô há mais de duas horas ensaiando uma recaída ...,ô há mais de duas horas ensaiando uma recaída ...
3,/jorge-e-mateus/,368,ela queima o arroz quebra copo na pia tropeça ...,ela queima o arroz tropeça no sofá machuca o d...
4,/aline-barros/,443,mestre eu preciso de um milagre transforma min...,mestre eu preciso de um milagre transforma min...


In [9]:
#removes stopwords and creates new column
df1['no_stopwords'] = df1['lyrics'].apply(lambda x: remove_stopwords(x.split(' ')))

In [10]:
#applies stemmer and creates new column
df1['orengo'] = df1['lyrics'].apply(lambda x: orengo(x.split(' ')))

In [11]:
#total number of words
df1['words'] = df1['lyrics'].apply(lambda x: len(x.split(' ')))

In [12]:
#total number of words without considering stopwords
df1['words_ns'] = df1['no_stopwords'].apply(lambda x: len(x))

In [13]:
#removes stopwords from translated text and creates new column
df1['ns_pt'] = df1['translated'].apply(lambda x: remove_stopwords(x.split(' ')))

In [14]:
#vocabulary
df1['unique'] = df1['lyrics'].apply(lambda x: len(set(x.split(' '))))

In [15]:
#vocabulary after stemming
df1['orengo_unique'] = df1['orengo'].apply(lambda x: len(set(x)))

In [16]:
#vocabulary without considering stopwords
df1['ns_unique'] = df1['no_stopwords'].apply(lambda x: len(set(x)))

In [17]:
#vocabulary without considering stopwords after stemming
df1['ns_orengo_unique'] = df1['no_stopwords'].apply(lambda x: len(set(orengo(x))))

In [18]:
#vocabulary after translation
df1['unique_pt'] = df1['translated'].apply(lambda x: len(set(x.split(' '))))

In [19]:
#vocabulary after stemming + translation
df1['orengo_pt'] = df1['translated'].apply(lambda x: len(set(orengo(x.split(' ')))))

In [20]:
#vocabulary without considering stopwords (translated)
df1['ns_uni_pt'] = df1['ns_pt'].apply(lambda x: len(set(x)))

In [21]:
#vocabulary without considering stopwords after stemming + translation
df1['ns_or_pt'] = df1['ns_pt'].apply(lambda x: len(set(orengo(x))))

In [22]:
df1.head()

,artist,foreign,lyrics,translated,no_stopwords,orengo,words,words_ns,ns_pt,unique,orengo_unique,ns_unique,ns_orengo_unique,unique_pt,orengo_pt,ns_uni_pt,ns_or_pt
0,/marilia-mendonca/,412,sei bem o que te faz bem eu sei mas no fundo e...,sei bem o que te faz bem eu sei mas no fundo e...,"[sei, bem, faz, bem, sei, fundo, tentei, falto...","[sei, bem, o, que, te, faz, bem, eu, sei, mas,...",3951,2165,"[sei, bem, faz, bem, sei, fundo, tentei, falto...",817,602,735,545,762,564,682,508
1,/roberto-carlos/,512,eu tenho tanto pra lhe falar mas com palavras ...,eu tenho tanto pra lhe falar mas com palavras ...,"[tanto, pra, falar, palavras, sei, dizer, gran...","[eu, tenh, tant, pra, lhe, fal, mas, com, pala...",5171,2710,"[tanto, pra, falar, palavras, sei, dizer, gran...",1040,730,943,667,981,693,884,628
2,/gusttavo-lima/,320,ô há mais de duas horas ensaiando uma recaída ...,ô há mais de duas horas ensaiando uma recaída ...,"[ô, duas, horas, ensaiando, recaída, número, t...","[ô, há, mais, de, dua, hor, ensai, uma, recaíd...",4414,2306,"[ô, duas, horas, ensaiando, recaída, número, t...",844,679,752,613,794,641,704,576
3,/jorge-e-mateus/,368,ela queima o arroz quebra copo na pia tropeça ...,ela queima o arroz tropeça no sofá machuca o d...,"[queima, arroz, quebra, copo, pia, tropeça, so...","[ela, queim, o, arroz, quebr, cop, na, pia, tr...",4120,2162,"[queima, arroz, tropeça, sofá, machuca, dedinh...",766,592,682,530,709,556,625,494
4,/aline-barros/,443,mestre eu preciso de um milagre transforma min...,mestre eu preciso de um milagre transforma min...,"[mestre, preciso, milagre, transforma, vida, e...","[mestr, eu, precis, de, um, milagr, transform,...",3826,2081,"[mestre, preciso, milagre, transforma, vida, e...",747,581,650,520,677,522,584,464


In [23]:
df1.to_csv('./data/final_data.csv', index=False)